In [1]:
import pandas as pd
from textblob import TextBlob

In [2]:
#read in the data, remove NaNs, reset index
data = pd.read_csv("review_final.csv")
data = data[pd.notnull(data['text'])]
data = data.reset_index(drop=True)

/Users/Francine/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
#decode review text to prevent decoding errors when breaking it into sentences
def decodeArray(text):
    try:
        return text.decode('utf-8','replace')
    except:
        return 0
    
data.text = data.text.apply(decodeArray)

In [4]:
#break review text into sentences
#this step literally took 24 hours
for i in range(0,len(data)):
    data.text[i] = TextBlob(data.text[i]).raw_sentences
    if i%10000 == 0:
        print("Processing the %dth review, %d more to go... Please be patient..." % (i,len(data)-i))

Processing the 0th review, 575541 more to go... Please be patient...
Processing the 10000th review, 565541 more to go... Please be patient...
Processing the 20000th review, 555541 more to go... Please be patient...
Processing the 30000th review, 545541 more to go... Please be patient...
Processing the 40000th review, 535541 more to go... Please be patient...
Processing the 50000th review, 525541 more to go... Please be patient...
Processing the 60000th review, 515541 more to go... Please be patient...
Processing the 70000th review, 505541 more to go... Please be patient...
Processing the 80000th review, 495541 more to go... Please be patient...
Processing the 90000th review, 485541 more to go... Please be patient...
Processing the 100000th review, 475541 more to go... Please be patient...
Processing the 110000th review, 465541 more to go... Please be patient...
Processing the 120000th review, 455541 more to go... Please be patient...
Processing the 130000th review, 445541 more to go...

/Users/Francine/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
#convert each sentence into a dataframe row preserving all other column information
data_sentences = pd.concat([pd.DataFrame(dict(zip(data.columns,data.ix[i]))) for i in range(len(data))], ignore_index=True)

In [6]:
#5,424,516 sentences from 575,541 reviews
len(data_sentences)

5424516

In [7]:
data_sentences.head()

,Unnamed: 0,business_id,date,review_id,stars,text,type,user_id,votes
0,0,4bEjOyTaDG24SY5TxsaUNQ,2005-06-23,UpKdXO3jEElnqtsWjYxQ2w,5,"Excellent food, great atmosphere, a bit noisy.",review,LAOHN2Z6INbwuGa3fx9UTw,"{u'funny': 0, u'useful': 0, u'cool': 0}"
1,0,4bEjOyTaDG24SY5TxsaUNQ,2005-06-23,UpKdXO3jEElnqtsWjYxQ2w,5,$$,review,LAOHN2Z6INbwuGa3fx9UTw,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,If you enjoy a little people watching with you...,review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
3,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,As the restaurant is directly across from the ...,review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
4,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,"The food fits the setting, with a traditional ...",review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"


In [8]:
#save as csv file
data_sentences.to_csv('review_sentences_final_challenge.csv', encoding='utf-8')

In [30]:
#service words selected from:
#1) http://www.vladsandulescu.com/topic-prediction-lda-user-reviews/
#2) Doug's NMF run
#3) Sudeep's slides: http://www.slideshare.net/SudeepDasPhD/chicago-june19sudeepdas

#service_words = ['manage', 'train', 'treat', ' time ', 'minute', 'service', 'waited', 'wait ', 'table', 'friendly',
#                 'waitress', 'waiter', 'job', 'work', 'experience', 'staff', 'lady', ' guy ', 'girl', 'person ',
#                 'customer', 'attention', 'detail', 'manager', 'employee', 'line', 'ask', 'server',
#                 'slow', 'fast', 'quick', 'seat', 'help', 'attend', 'recommend', 'suggestion', 'forgot', 'forget',
#                 'professional', 'knowledge', 'prompt']

service_words = ['management', 'manager', 'waitress', 'waiter', 'staff', 'employee', 'cashier', 'server', 'guy', 'customer', #people nouns
                 'service', 'counter', 'minute', 'attention', 'knowledge', 'nerve', #other nouns
                 'greet', 'seated', 'taking', 'treated', 'waited', 'forgot', 'apologize', #verbs
                 'friendly', 'professional', 'prompt', 'attentive', 'helpful', 'slow', 'rude', 'untrained', 'unattended'] #adjectives

In [31]:
len(service_words)

32

In [32]:
#select sentences containing only the service words
all_data = []
for i in service_words:
    all_data.append(data_sentences[data_sentences['text'].str.contains(i)])

In [33]:
#concaternate all dataframes, sort, drop duplicate rows and reset index
test_data = pd.concat(all_data)
test_data = test_data.sort().drop_duplicates()
test_data = test_data.reset_index(drop=True)

In [35]:
test_data.text[1]

u'We had excellent service..polite, prompt, helpful (great at helping us pick a great wine suited to our selections!)'

In [36]:
#number of service-related sentences
len(test_data)

646123

In [37]:
#save as csv file
test_data.to_csv('review_sentences_final_challenge_service_words_NEW.csv', encoding='utf-8')

In [38]:
servicesentences = pd.read_csv('review_sentences_final_challenge_service_words_NEW.csv')

In [39]:
servicesentences.head()

,Unnamed: 0,Unnamed: 0.1,business_id,date,review_id,stars,text,type,user_id,votes
0,0,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,As the restaurant is directly across from the ...,review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
1,1,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,"We had excellent service..polite, prompt, help...",review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,2,1,4bEjOyTaDG24SY5TxsaUNQ,2005-10-10,qYJV5kIViwer1P1Cb2OSfw,5,"And by the way, the wait staff aren't above ...",review,H43f_rD3czq0UqY-4zhWjA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
3,3,5,4bEjOyTaDG24SY5TxsaUNQ,2006-01-31,M6U9acJkmCXZkEvIIDFtfQ,5,"Secondly, the service was OUTSTANDING.",review,DeBP40aPIRv3h1ED17X4Qg,"{u'funny': 1, u'useful': 0, u'cool': 0}"
4,4,5,4bEjOyTaDG24SY5TxsaUNQ,2006-01-31,M6U9acJkmCXZkEvIIDFtfQ,5,The waitress even put in a special request for...,review,DeBP40aPIRv3h1ED17X4Qg,"{u'funny': 1, u'useful': 0, u'cool': 0}"
